In [57]:
import numpy as np
import random 
import pandas as pd

import ortools
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

def create_random_distance_matrix(n):
# Creating a distance matrix that follows the triangle inequality
    matrix = np.zeros((n, n), dtype=int)

    for i in range(n):
        for j in range(i + 1, n):
            matrix[i, j] = random.randint(1, 100)

    for i in range(n):
        for j in range(i + 1, n):
            for k in range(j + 1, n):
                matrix[j, k] = random.randint(1, 100)
                while matrix[i, j] + matrix[j, k] < matrix[i, k]:
                    matrix[i, k] = random.randint(1, 100)

    matrix = matrix + matrix.T

  
    return matrix

In [58]:
matrix_10 = create_random_distance_matrix(10)
matrix_20 = create_random_distance_matrix(20)

In [67]:
class Optimization:

    def __init__(self, data, num_vehicles):
        depot = 0
        self.data = {}
        self.data['distance_matrix'] = data
        self.data['num_vehicles'] = num_vehicles
        self.data['depot'] = depot

        self.manager = pywrapcp.RoutingIndexManager(
            len(self.data["distance_matrix"]), self.data["num_vehicles"], self.data["depot"]
        )
        self.routing = pywrapcp.RoutingModel(self.manager)
        self.transit_callback_index = self.routing.RegisterTransitCallback(self.distance_callback)

        self.routing.SetArcCostEvaluatorOfAllVehicles(self.transit_callback_index)

        self.search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        self.search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
        )
        self.solution = self.routing.SolveWithParameters(self.search_parameters)

    def print_solution(self):
        """Prints solution on console."""
        print(f"Objective: {self.solution.ObjectiveValue()} miles")
        index = self.routing.Start(0)
        plan_output = "Route for vehicle 0:\n"
        route_distance = 0
        while not self.routing.IsEnd(index):
            plan_output += f" {self.manager.IndexToNode(index)} ->"
            previous_index = index
            index = self.solution.Value(self.routing.NextVar(index))
            route_distance += self.routing.GetArcCostForVehicle(previous_index, index, 0)
        plan_output += f" {self.manager.IndexToNode(index)}\n"
        print(plan_output)
        plan_output += f"Route distance: {route_distance}miles\n"
        
        routes = []
        for route_nbr in range(self.routing.vehicles()):
            index = self.routing.Start(route_nbr)
            route = [self.manager.IndexToNode(index)]
            while not self.routing.IsEnd(index):
                index = self.solution.Value(self.routing.NextVar(index))
                route.append(self.manager.IndexToNode(index))
            routes.append(route)
        
        for i, route in enumerate(routes):
            print('Route', i, route)

    def distance_callback(self, from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = self.manager.IndexToNode(from_index)
        to_node = self.manager.IndexToNode(to_index)
        return self.data["distance_matrix"][from_node][to_node]

In [68]:
matrix_10

array([[ 0, 22, 76,  5,  5,  2,  7, 12,  3, 10],
       [22,  0, 68, 12, 28, 21, 20,  8, 10, 15],
       [76, 68,  0, 24, 15, 29, 37, 19, 34, 21],
       [ 5, 12, 24,  0, 23, 35, 60, 11, 43, 28],
       [ 5, 28, 15, 23,  0, 29, 39, 15, 11,  5],
       [ 2, 21, 29, 35, 29,  0, 30,  2, 27, 25],
       [ 7, 20, 37, 60, 39, 30,  0,  3,  7, 34],
       [12,  8, 19, 11, 15,  2,  3,  0,  4, 42],
       [ 3, 10, 34, 43, 11, 27,  7,  4,  0, 51],
       [10, 15, 21, 28,  5, 25, 34, 42, 51,  0]])

In [69]:
opt = Optimization(matrix_10, num_vehicles=1)

opt.print_solution()

Objective: 88 miles
Route for vehicle 0:
 0 -> 3 -> 2 -> 4 -> 9 -> 1 -> 8 -> 6 -> 7 -> 5 -> 0

Route 0 [0, 3, 2, 4, 9, 1, 8, 6, 7, 5, 0]


In [70]:
opt = Optimization(matrix_10, num_vehicles=3)

opt.print_solution()

Objective: 88 miles
Route for vehicle 0:
 0 -> 0

Route 0 [0, 0]
Route 1 [0, 0]
Route 2 [0, 3, 2, 4, 9, 1, 8, 6, 7, 5, 0]


In [71]:
matrix_20

array([[ 0, 89, 65, 92, 25, 15, 25,  8, 14,  6, 19,  2, 10, 22,  3, 27,
        11, 29,  9, 30],
       [89,  0, 18, 64, 26,  7, 24,  8,  2,  3, 16, 15, 22,  5,  6,  6,
         2,  9,  2,  7],
       [65, 18,  0, 52, 34, 49, 35, 38, 54,  9, 25, 36, 22, 17, 13, 18,
         3,  8,  3,  4],
       [92, 64, 52,  0, 29,  3,  4, 20, 16, 23, 10, 16, 40, 22, 17,  3,
        32,  9,  8,  1],
       [25, 26, 34, 29,  0, 91, 70, 74, 81, 41, 29, 29, 41, 16, 66, 11,
        23, 35,  7,  5],
       [15,  7, 49,  3, 91,  0, 93, 30, 46, 14, 58, 50, 30, 41, 18, 18,
        17,  7, 12, 31],
       [25, 24, 35,  4, 70, 93,  0, 26, 80, 32, 40, 18,  8, 22, 21, 38,
        20,  1, 34, 38],
       [ 8,  8, 38, 20, 74, 30, 26,  0, 83, 54, 87, 25, 86, 46, 30, 42,
        28,  4, 48, 24],
       [14,  2, 54, 16, 81, 46, 80, 83,  0, 29,  1, 18, 16, 19, 14,  7,
        11, 15, 46,  8],
       [ 6,  3,  9, 23, 41, 14, 32, 54, 29,  0,  9, 23,  2, 27, 14, 15,
         5,  1,  2, 14],
       [19, 16, 25, 10, 29, 58

In [72]:
opt = Optimization(matrix_20, num_vehicles=1)

opt.print_solution()

Objective: 105 miles
Route for vehicle 0:
 0 -> 7 -> 17 -> 6 -> 12 -> 9 -> 10 -> 8 -> 15 -> 3 -> 5 -> 18 -> 4 -> 19 -> 2 -> 16 -> 1 -> 13 -> 11 -> 14 -> 0

Route 0 [0, 7, 17, 6, 12, 9, 10, 8, 15, 3, 5, 18, 4, 19, 2, 16, 1, 13, 11, 14, 0]


In [73]:
opt = Optimization(matrix_20, num_vehicles=3)

opt.print_solution()

Objective: 109 miles
Route for vehicle 0:
 0 -> 0

Route 0 [0, 0]
Route 1 [0, 7, 0]
Route 2 [0, 11, 13, 1, 15, 8, 10, 9, 12, 6, 17, 5, 3, 19, 4, 18, 2, 16, 14, 0]
